# HW #6: Evaluating SVM
---

In this HW, you'll explore several data sets with SVM classifiers and compare them to logistic regression classifiers.  

Use markdown cells to explain your observation.  

We will use two data sets:

**Breast Cancer**

    breast_cancer.csv

**Car Evaluation**

    car.csv


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model, datasets
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

plt.style.use('fivethirtyeight')

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1kxV5c1FgaUM-KFoHVAM7-np6LnpIFZxQ'}) # replace the id with id of file you want to access
downloaded.GetContentFile('breast_cancer.csv')


downloaded = drive.CreateFile({'id':'1ILf9_rzNvU9ip4_tyQx3oVSrMciMg_sm'}) # replace the id with id of file you want to access
downloaded.GetContentFile('car.csv')

### 1) Load the breast cancer data.

- Are there any missing values? If so, impute or clean the data set.
- Select a classification target and predictors.

In [0]:
df = pd.read_csv('breast_cancer.csv', na_values='?')

In [0]:
# A:

df.head()

,Sample_code_number,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [0]:
df.dtypes

Sample_code_number               int64
Clump_Thickness                  int64
Uniformity_of_Cell_Size          int64
Uniformity_of_Cell_Shape         int64
Marginal_Adhesion                int64
Single_Epithelial_Cell_Size      int64
Bare_Nuclei                    float64
Bland_Chromatin                  int64
Normal_Nucleoli                  int64
Mitoses                          int64
Class                            int64
dtype: object

In [0]:
#df.info()

df.isnull().sum()

## Yes,there are missing values in the data set for the column Bare_Nucleui

Sample_code_number              0
Clump_Thickness                 0
Uniformity_of_Cell_Size         0
Uniformity_of_Cell_Shape        0
Marginal_Adhesion               0
Single_Epithelial_Cell_Size     0
Bare_Nuclei                    16
Bland_Chromatin                 0
Normal_Nucleoli                 0
Mitoses                         0
Class                           0
dtype: int64

In [0]:


#df[df.isnull().any(axis=1)]

df[df.isnull().any(axis=1)] ## This gives the records those are are null values in the dataset



,Sample_code_number,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
23,1057013,8,4,5,1,2,NaN,7,3,1,4
40,1096800,6,6,6,9,6,NaN,7,8,1,2
139,1183246,1,1,1,1,1,NaN,2,1,1,2
145,1184840,1,1,3,1,2,NaN,2,1,1,2
158,1193683,1,1,2,1,3,NaN,1,1,1,2
164,1197510,5,1,1,1,2,NaN,3,1,1,2
235,1241232,3,1,4,1,2,NaN,3,1,1,2
249,169356,3,1,1,1,2,NaN,3,1,1,2
275,432809,3,1,3,1,2,NaN,2,1,1,2
292,563649,8,8,8,1,2,NaN,6,10,1,4


In [0]:
 df.dropna(inplace = True) ############ by dropping null values

In [0]:
df[df.isnull().any(axis=1)] ## This gives the records those are are null values in the dataset


,Sample_code_number,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class


In [0]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(XX)
XXX = scaler.transform(XX)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [0]:
XX.columns

Index(['Clump_Thickness', 'Uniformity_of_Cell_Size',
       'Uniformity_of_Cell_Shape', 'Marginal_Adhesion',
       'Single_Epithelial_Cell_Size', 'Bare_Nuclei', 'Bland_Chromatin',
       'Normal_Nucleoli', 'Mitoses'],
      dtype='object')

In [0]:
# data = pd.DataFrame({})

features = pd.DataFrame(XXX, columns = XX.columns)

features.head()



,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses
0,0.197905,-0.702212,-0.741774,-0.639366,-0.555608,-0.698853,-0.181827,-0.612927,-0.3484
1,0.197905,0.277252,0.262783,0.758032,1.695166,1.772867,-0.181827,-0.285105,-0.3484
2,-0.511643,-0.702212,-0.741774,-0.639366,-0.555608,-0.424217,-0.181827,-0.612927,-0.3484
3,0.552679,1.583204,1.602192,-0.639366,-0.105454,0.125054,-0.181827,1.354008,-0.3484
4,-0.156869,-0.702212,-0.741774,0.059333,-0.555608,-0.698853,-0.181827,-0.612927,-0.3484


In [0]:
df.isnull().sum()

Sample_code_number             0
Clump_Thickness                0
Uniformity_of_Cell_Size        0
Uniformity_of_Cell_Shape       0
Marginal_Adhesion              0
Single_Epithelial_Cell_Size    0
Bare_Nuclei                    0
Bland_Chromatin                0
Normal_Nucleoli                0
Mitoses                        0
Class                          0
dtype: int64

In [0]:
## Here there are 2-458 classes and 4 - 241 classes in  the dataset.
df.Class.value_counts()

#df.Sample_code_number.value_counts()

2    444
4    239
Name: Class, dtype: int64

### 2) Build an SVM classifier on the data.

For details on the SVM classifier, [click here](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

- Initialize and train a linear SVM. What's the average accuracy score with a five-fold cross-validation?
- Repeat using a radial basis function (RBF) classifier. Compare the scores. Which one is better?
- Print a confusion matrix and classification report for your best model using training and testing data.

**[View Classification report and Confusion matrix coding example here](http://joshlawman.com/metrics-classification-report-breakdown-precision-recall-f1/)**

In [0]:
## Convert the target label into dummy using label encoder

from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(df['Class']) # Fit transform gives as 1,2,3,4 instead of just 1,0 and 1,0 and 1,0 
X = df.drop('Class', axis=1)  

X.columns

Index(['Sample_code_number', 'Clump_Thickness', 'Uniformity_of_Cell_Size',
       'Uniformity_of_Cell_Shape', 'Marginal_Adhesion',
       'Single_Epithelial_Cell_Size', 'Bare_Nuclei', 'Bland_Chromatin',
       'Normal_Nucleoli', 'Mitoses'],
      dtype='object')

In [0]:
XX = X.drop('Sample_code_number', axis=1)

XX.shape

(683, 9)

In [0]:
# A:

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model, datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score



########

from sklearn.model_selection import KFold
from sklearn import metrics

svc_modell = SVC(kernel = 'linear')
scores = cross_val_score(svc_modell,XX,Y,cv = 5)

print('mean scores: ',np.mean(scores))
print('std of scores', np.std(scores))

print('scores:', scores)

mean scores:  0.9663801027304677
std of scores 0.014975650493662142
scores: [0.94890511 0.94890511 0.97810219 0.97080292 0.98518519]


In [0]:
### Building model using "rbf" Kernel

svc_model = SVC(kernel = 'rbf')
scores = cross_val_score(svc_model,XX,Y,cv = 5)

print('mean scores: ',np.mean(scores))
print('std of scores', np.std(scores))
print('scores:', scores)

mean scores:  0.9503217085698837
std of scores 0.0349323662284763
scores: [0.90510949 0.91240876 0.96350365 0.98540146 0.98518519]


#### The mean SCORE for an SVM with linear  kernel has an accuracy of 0.96638 where as for Rbf has 0.9503.
#### When you compare both the models with different kernel function, svm with linear kernel has higher score compared to rbf

In [0]:
#### Confusion matrix

## Slplit data into test and train ##

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(XX, Y, test_size = 0.2,random_state = 20)  


In [0]:
# Building model using SVM with linear kernel ###

from sklearn.svm import SVC  
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_train, y_train)  


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
# Applying prediction on test data 

y_pred = svclassifier.predict(X_test)  


In [0]:

##### Model Evaluation using by Applying confusion Matrix ###
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  



[[94  5]
 [ 2 36]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.96        99
           1       0.88      0.95      0.91        38

   micro avg       0.95      0.95      0.95       137
   macro avg       0.93      0.95      0.94       137
weighted avg       0.95      0.95      0.95       137



In [0]:
from sklearn.metrics import confusion_matrix

final_svc_poly = SVC(kernel='linear')
final_svc_poly.fit(X_train, y_train)  

confusion_mat = confusion_matrix(y_test, final_svc_poly.predict(X_test))


In [0]:
from sklearn.metrics import confusion_matrix
import pandas as pd

confusion_df = pd.DataFrame(confusion_matrix(y_test, final_svc_poly.predict(X_test)),
             columns=["Predicted Class " + str(class_name) for class_name in [0,1]],
             index = ["Class " + str(class_name) for class_name in [0,1]])

print(confusion_df)


         Predicted Class 0  Predicted Class 1
Class 0                 94                  5
Class 1                  2                 36


#### 2.A) Are there more false positives or false negatives? Is this good or bad?

In [0]:
# A: ## 

# In a binary classification problem, Class 0 is considered as False and Class 1 as True.
# Now, When compared to both False positives and False Negatives. There are more number of false positives.
# In our case, in breast cancer problem. We got less number of false negative it is good. But we should try to minimize it further if possible.
# There are more number of false positive. Which means, a patient had no cancer but model predicted  as he had cancer.
# This is good but the model has more in number for type-1 error.

### 3) Compare SVM and logistic regression using cars' dataset.

You should work on the following:

- With default hyperparameters, compare the performance of SVM vs. Logistic Regression
- What choice of kernel would give SVM the best performance

Use the following to discuss performance
- The mean value of cross-validate scores.
- Examine confusion matrices and classification reports.


In [0]:
car = pd.read_csv('car.csv')
car.head(2)

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc


In [0]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying           1728 non-null object
maint            1728 non-null object
doors            1728 non-null object
persons          1728 non-null object
lug_boot         1728 non-null object
safety           1728 non-null object
acceptability    1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


In [0]:
# A:

from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(car['acceptability'])

x = pd.get_dummies(car.drop(['acceptability'],axis = 1))

#car.sample(3)
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,Y,test_size=0.3,random_state= 42)


In [0]:


logreg = LogisticRegression()
print("Logistic Regression:",cross_val_score(logreg, x,Y, cv=5, scoring='accuracy').mean())

svc = SVC(kernel = 'linear')        # linear
print( "SVM with linear kernel:",cross_val_score(svc, x, Y, cv=5, scoring='accuracy').mean())

poly_svc = SVC(kernel = 'poly')#, degree = 1)        #  polys

print("SVM with poly kernel:",cross_val_score(poly_svc, x,Y, cv=5, scoring='accuracy').mean())


rbf_svc = SVC(kernel = 'rbf')#,C= 0.1)           # rbf
print("SVM with rbf kernel:",cross_val_score(rbf_svc, x, Y, cv=5, scoring='accuracy').mean())


Logistic Regression: 0.7586537168974324
SVM with linear kernel: 0.8033573060895283
SVM with poly kernel: 0.7002352466729399
SVM with rbf kernel: 0.7511258233633553


In [0]:
### We can see from above models that SVM with linear kernel performed well,which has an accuracy over 0.803 where as logistic regression has 0.75

In [0]:
from sklearn.model_selection import cross_val_predict
y_pred1=cross_val_predict(svc,x_test,y_test)

In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("Precision score: {}".format(precision_score(y_test,y_pred1,average='micro')))

print("Recall score: {}".format(recall_score(y_test,y_pred1,average='micro')))

print("F1 Score: {}".format(f1_score(y_test,y_pred1,average='micro')))



Precision score: 0.9036608863198459
Recall score: 0.9036608863198459
F1 Score: 0.9036608863198459


In [0]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.77      0.84      0.80       118
           1       0.62      0.53      0.57        19
           2       0.97      0.95      0.96       358
           3       0.88      0.88      0.88        24

   micro avg       0.90      0.90      0.90       519
   macro avg       0.81      0.80      0.80       519
weighted avg       0.91      0.90      0.90       519



In [0]:
from sklearn.metrics import confusion_matrix
import pandas as pd
confusion_matrix(y_test,y_pred1)

array([[ 99,   6,  12,   1],
       [  7,  10,   0,   2],
       [ 19,   0, 339,   0],
       [  3,   0,   0,  21]])

In [0]:
confusion_df = pd.DataFrame(confusion_matrix(y_test,y_pred1),
           columns=["Predicted Class " + str(class_name) for class_name in [0,1,2,3]],
           index = ["Class " + str(class_name) for class_name in [0,1,2,3]])

print(confusion_df)

         Predicted Class 0  Predicted Class 1  Predicted Class 2  \
Class 0                 99                  6                 12   
Class 1                  7                 10                  0   
Class 2                 19                  0                339   
Class 3                  3                  0                  0   

         Predicted Class 3  
Class 0                  1  
Class 1                  2  
Class 2                  0  
Class 3                 21  


In [0]:
# In recall metric, the fraction of positives, were identified as 0.4 for one class and the rest of them has 0.53 with weighted average of 0.9
# The F1 score is greater than 70% for 3 classes and 48% for one but the weighted average of 0.56
# The accuracy of positive predictions is more than 60% for all classes in linear SVM kernel, with an weighted  average over 0.91
# From Our confusion matrix, our model has predicted class 2 more accurately compared to rest other classes